In [1]:
import pandas as pd
import numpy as np

# Đọc file CSV đã lấy về ở bước trước
df = pd.read_csv('../data/logs.csv')

# Hiển thị 5 dòng đầu tiên để kiểm tra
print("Data sample:")
display(df.head())

# In thông tin tổng quan về các cột và kiểu dữ liệu
print("\nData Info:")
df.info()

Data sample:


,status,log,geoip,request_uri,useragent,body_bytes_sent,@version,@timestamp,tags,event,host,http_referer,http_user_agent,remote_addr,request_method
0,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/about-us,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,2025-10-17T14:24:16.000Z,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",{'name': 'a0478df4e7c0'},NaN,python-requests/2.31.0,172.18.0.1,GET
1,200,{'file': {'path': '/var/log/nginx/access.log'}},{},/index.html,"{'device': {'name': 'Other'}, 'os': {'full': '...",615,1,2025-10-17T14:24:16.000Z,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",{'name': 'a0478df4e7c0'},NaN,python-requests/2.31.0,172.18.0.1,GET
2,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/products,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,2025-10-17T14:24:26.000Z,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",{'name': 'a0478df4e7c0'},NaN,python-requests/2.31.0,172.18.0.1,GET
3,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/about-us,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,2025-10-17T14:24:26.000Z,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",{'name': 'a0478df4e7c0'},NaN,python-requests/2.31.0,172.18.0.1,GET
4,200,{'file': {'path': '/var/log/nginx/access.log'}},{},/index.html,"{'device': {'name': 'Other'}, 'os': {'full': '...",615,1,2025-10-17T14:24:10.000Z,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",{'name': 'a0478df4e7c0'},NaN,python-requests/2.31.0,172.18.0.1,GET



Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   status           268 non-null    int64  
 1   log              268 non-null    object 
 2   geoip            268 non-null    object 
 3   request_uri      268 non-null    object 
 4   useragent        268 non-null    object 
 5   body_bytes_sent  268 non-null    int64  
 6   @version         268 non-null    int64  
 7   @timestamp       268 non-null    object 
 8   tags             268 non-null    object 
 9   event            268 non-null    object 
 10  host             268 non-null    object 
 11  http_referer     0 non-null      float64
 12  http_user_agent  268 non-null    object 
 13  remote_addr      268 non-null    object 
 14  request_method   268 non-null    object 
dtypes: float64(1), int64(3), object(11)
memory usage: 31.5+ KB


In [2]:
# Chuyển cột @timestamp sang kiểu datetime...
df['@timestamp'] = pd.to_datetime(df['@timestamp'])

# Kiểm tra xem cột 'geoip.city_name' có tồn tại không
if 'geoip.city_name' in df.columns:
    # Nếu có, thì mới điền các giá trị thiếu
    df['geoip.city_name'] = df['geoip.city_name'].fillna('Unknown')
else:
    # Nếu không có, hãy tạo ra cột đó với một giá trị mặc định
    # Điều này đảm bảo các bước sau không bị lỗi
    print("Column 'geoip.city_name' not found. Creating it with default value 'Unknown'.")
    df['geoip.city_name'] = 'Unknown'

print("Cleaned data and converted '@timestamp' to datetime format.")

Column 'geoip.city_name' not found. Creating it with default value 'Unknown'.
Cleaned data and converted '@timestamp' to datetime format.


In [3]:
# Đặt @timestamp làm index để thực hiện gom nhóm theo thời gian
# Đảm bảo df có index là timestamp trước
if not isinstance(df.index, pd.DatetimeIndex):
    df = df.set_index('@timestamp')
df = df.sort_index()

# Tạo các đặc trưng bằng cách gom nhóm theo IP và cửa sổ thời gian 1 phút
# group_keys=False là để tránh thêm một tầng index không cần thiết
grouped = df.groupby(['remote_addr', pd.Grouper(freq='1min')]).agg(
    request_count=('request_uri', 'count'),          
    unique_uri_count=('request_uri', 'nunique'),     
    http_404_count=('status', lambda x: (x == 404).sum()),
    http_401_count=('status', lambda x: (x == 401).sum()) 
)

# Chuyển các index (remote_addr và @timestamp) thành các cột thông thường
features_df = grouped.reset_index()

print("Feature calculation completed. Sample of features:")
display(features_df.head())

print("\nColumns of features_df:")
print(features_df.columns)

Feature calculation completed. Sample of features:


,remote_addr,@timestamp,request_count,unique_uri_count,http_404_count,http_401_count
0,172.18.0.1,2025-10-17 14:24:00+00:00,29,4,18,0
1,172.18.0.1,2025-10-17 14:25:00+00:00,106,64,105,0
2,172.18.0.1,2025-10-17 14:59:00+00:00,1,1,0,0
3,172.18.0.1,2025-10-17 15:00:00+00:00,32,4,17,0
4,172.18.0.1,2025-10-17 15:01:00+00:00,100,61,100,0



Columns of features_df:
Index(['remote_addr', '@timestamp', 'request_count', 'unique_uri_count',
       'http_404_count', 'http_401_count'],
      dtype='object')


In [5]:
# Đảm bảo index là cột timestamp
if not isinstance(df.index, pd.DatetimeIndex):
    df = df.set_index('@timestamp')

# Tạo cột time_window trong df gốc
df['time_window'] = df.index.floor('min')

# Đổi tên cột '@timestamp' trong features_df thành 'time_window' để khớp nhau
features_df.rename(columns={'@timestamp': 'time_window'}, inplace=True)
# -------------------

# Bây giờ, gộp hai DataFrame lại với nhau, cả hai đều đã có cột 'time_window'
final_df = pd.merge(df.reset_index(), features_df, on=['remote_addr', 'time_window'], how='left')

# Điền 0 cho các giá trị NaN có thể phát sinh sau khi gộp
final_df.fillna(0, inplace=True)

print("Merged features back into the main DataFrame. Final sample:")
display(final_df.head())

Merged features back into the main DataFrame. Final sample:


,@timestamp,status,log,geoip,request_uri,useragent,body_bytes_sent,@version,tags,event,...,http_referer,http_user_agent,remote_addr,request_method,geoip.city_name,time_window,request_count,unique_uri_count,http_404_count,http_401_count
0,2025-10-17 14:24:08+00:00,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/about-us,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",...,0.0,python-requests/2.31.0,172.18.0.1,GET,Unknown,2025-10-17 14:24:00+00:00,29,4,18,0
1,2025-10-17 14:24:10+00:00,200,{'file': {'path': '/var/log/nginx/access.log'}},{},/index.html,"{'device': {'name': 'Other'}, 'os': {'full': '...",615,1,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",...,0.0,python-requests/2.31.0,172.18.0.1,GET,Unknown,2025-10-17 14:24:00+00:00,29,4,18,0
2,2025-10-17 14:24:13+00:00,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/about-us,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",...,0.0,python-requests/2.31.0,172.18.0.1,GET,Unknown,2025-10-17 14:24:00+00:00,29,4,18,0
3,2025-10-17 14:24:16+00:00,404,{'file': {'path': '/var/log/nginx/access.log'}},{},/about-us,"{'device': {'name': 'Other'}, 'os': {'full': '...",153,1,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",...,0.0,python-requests/2.31.0,172.18.0.1,GET,Unknown,2025-10-17 14:24:00+00:00,29,4,18,0
4,2025-10-17 14:24:16+00:00,200,{'file': {'path': '/var/log/nginx/access.log'}},{},/index.html,"{'device': {'name': 'Other'}, 'os': {'full': '...",615,1,['_geoip_lookup_failure'],"{'original': '{""@timestamp"":""2025-10-17T14:24:...",...,0.0,python-requests/2.31.0,172.18.0.1,GET,Unknown,2025-10-17 14:24:00+00:00,29,4,18,0


In [6]:
# Lưu dataframe đã có đầy đủ feature ra file mới
processed_output_path = '../data/processed_logs.csv'
final_df.to_csv(processed_output_path, index=False)

print(f"Processed data has been saved to '{processed_output_path}'")

Processed data has been saved to '../data/processed_logs.csv'
